<a href="https://colab.research.google.com/github/rkaclfdl123/TIL/blob/master/1126_fastAPI_best%EB%AA%A8%EB%8D%B8_%EC%A0%81%EC%9A%A9(%EA%B3%A0%EC%A4%80%EC%88%98).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/Astrophysics/1122_mount

/content/drive/.shortcut-targets-by-id/16oCueqwvnKt564dZ0WW_5p3nppsa07g8/Astrophysics/1122_mount


In [1]:
!pip install fastapi nest-asyncio pyngrok uvicorn

     |████████████████████████████████| 51 kB 772 kB/s 
     |████████████████████████████████| 745 kB 19.7 MB/s 
     |████████████████████████████████| 54 kB 3.7 MB/s 
     |████████████████████████████████| 61 kB 357 kB/s 
     |████████████████████████████████| 10.1 MB 60.2 MB/s 
     |████████████████████████████████| 78 kB 9.0 MB/s 
     |████████████████████████████████| 54 kB 3.6 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19006 sha256=e030926c8b3e8d88e770b3b47ff02ade2a0c5191fe86ead9ce17e8cf4803c91b
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [2]:
!pip install jinja2
!pip install aiofiles
!pip install python-multipart

  Created wheel for python-multipart: filename=python_multipart-0.0.5-py3-none-any.whl size=31678 sha256=659afb7634f7a1f46e4294cd9438879c3705f986af6e7bae037df18a8223ad36
  Stored in directory: /root/.cache/pip/wheels/2c/41/7c/bfd1c180534ffdcc0972f78c5758f89881602175d48a8bcd2c
Successfully built python-multipart


In [5]:
from typing import Optional
from fastapi import FastAPI
from fastapi import FastAPI, File, UploadFile, Request, Form, File, UploadFile
from fastapi import FastAPI, status
from fastapi.responses import HTMLResponse
from typing import List
import os
import uvicorn
from fastapi import APIRouter
from fastapi.staticfiles import StaticFiles
import nest_asyncio
from pyngrok import ngrok
from fastapi.templating import Jinja2Templates
from PIL import Image
from io import BytesIO
import torch

In [ ]:
app = FastAPI()

@app.get("/")
def home(request: Request):
	'''
	Returns barebones HTML form allowing the user to select a file and model
	'''

	html_content = '''
<form method="post" enctype="multipart/form-data">
  <div>
    <label>Upload Image</label>
    <input name="file" type="file" multiple>
    <div>
      <label>Select YOLO Model</label>
      <select name="model_name">
        <option>yolov5s</option>
        <option>yolov5m</option>
        <option>yolov5l</option>
        <option>yolov5x</option>
      </select>
    </div>
  </div>
  <button type="submit">Submit</button>
</form>
'''

	return HTMLResponse(content=html_content, status_code=200)

def results_to_json(results, model):
    ''' Helper function for process_home_form()'''
    return [
        [
            {
                "class": int(pred[5]),
                "class_name": model.model.names[int(pred[5])],
                "normalized_box": pred[:4].tolist(),
                "confidence": float(pred[4]),
            }
            for pred in result
        ]
        #switch this to results.xyxy to get bbox pixels
        for result in results.xyxyn 
    ]

@app.post("/")
async def process_home_form(file: UploadFile = File(...), 
							model_name: str = Form(...)):
	
	'''
	Requires an image file upload, model name (ex. yolov5s).
	Returns: json response with list of list of dicts.
		Each dict contains class, class_name, confidence, normalized_bbox
	'''
#이부분 path 경로설정만 하셔요
	model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/drive/MyDrive/Astrophysics/1122_mount/yolov5/runs/train/multiple_yolov5s_results/weights/best.pt')

  #This is how you decode + process image with PIL
	results = model( Image.open(BytesIO(await file.read())))
    

  #This is how you decode + process image with OpenCV
  #results = model(cv2.imdecode(np.fromstring(await file.read(), np.uint8), cv2.IMREAD_COLOR))

	json_results = results_to_json(results,model)
	return json_results

ngrok_tunnel = ngrok.connect(8000)
print('Public URL :' , ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, host='0.0.0.0', port=8000)

INFO:     Started server process [74]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL : http://ce02-35-230-80-163.ngrok.io
INFO:     222.99.3.47:0 - "GET / HTTP/1.1" 200 OK
INFO:     222.99.3.47:0 - "GET / HTTP/1.1" 200 OK
INFO:     222.99.3.47:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found


Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /root/.cache/torch/hub/master.zip


requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...


YOLOv5 🚀 2021-11-26 torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)



  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



Fusing layers... 
Model Summary: 213 layers, 7042489 parameters, 0 gradients
Adding AutoShape... 


INFO:     222.99.3.47:0 - "POST / HTTP/1.1" 200 OK
